In [2]:
from time import sleep
from MoxScraper.MoxScraper import MoxScraper
import pandas as pd

wants = [
    {
        'name': 'Wedding Announcement // Wedding Festivity',
        'qty': 2
    },
    # {
    #     'name': 'Thoughtseize',
    #     'qty': 5
    # },
    # {
    #     'name': 'Goblin Tomb Raider',
    #     'qty': 3
    # },
]

shipping_cost = {
        'D20Battleground': 90,
        'Greedy Gold': 100,
        'Sword & Board': 80,
        'Luckshack': 120,
        'Dracoti': 90,
        'The Warren': 0,
        'The Stone Dragon': 100,
        'Mirage Gaming': 100,
}

return_df = pd.DataFrame()
scraper = MoxScraper()

for item in wants:
    df = scraper.format_for_retailer(scraper.scrape_mox_df(item['name']))
    if not df.empty:
        return_df = pd.concat([return_df, df], ignore_index=True)

    sleep(5)
print(return_df)
card_df = return_df.copy()
# Combine 'Column1' and 'Column2' into a new column 'Combined'
card_df['name'] = return_df['name'].astype(str) + '-' + return_df['id'].astype(str)
card_df = card_df.drop('id', axis=1)
card_df['price'] = card_df['price'].str.replace('R','').astype(float)


# Convert to dictionary
cost = { (row['name'], row['retailer_name']): row['price'] for _, row in card_df.iterrows() }
stock = { (row['name'], row['retailer_name']): row['stock'] for _, row in card_df.iterrows() }


# Apply filter for available stock
cards = []
    
for want in wants:
    amount = sum(return_df[return_df["name"]==want['name']]["stock"])
    if amount < want['qty']:
        cards.append({
            'name': want['name'],
            'qty': amount
        })
    else:
        cards.append({
            'name': want['name'],
            'qty': want['qty']
        })

2024-07-20 11:28:28,097 [CUSTOM] [CustomLogger.py:26] - Querying for Wedding Announcement // Wedding Festivity
2024-07-20 11:28:29,320 [CUSTOM] [CustomLogger.py:26] - Response code: 200


KeyError: 'name'

In [ ]:
# return_df.head(100)
cards = []
for card in return_df['name'].unique():
    print(f'{card}: {sum(return_df[return_df["name"]==card]["stock"])}')
    
for want in wants:
    amount = sum(return_df[return_df["name"]==want['name']]["stock"])
    if amount < want['qty']:
        cards.append({
            'name': want['name'],
            'qty': amount
        })
    else:
        cards.append({
            'name': want['name'],
            'qty': want['qty']
        })
        
print(cards)

In [ ]:
import pulp as pl

problem = pl.LpProblem("Magic_Magic", pl.LpMinimize)

cards = [
    {
        'name': 'land',
        'qty': 2
    }
]

cost = {
    ('land', 'storea') : 6,
    ('land', 'storeb') : 80,
}

# Define shipping costs
shipping_cost = {
    'storea': 5,
    'storeb': 5,
}

M = 1000

selected_indicator = pl.LpVariable.dicts(
    "card_store_qty",
    # {('land','storea'),('land', 'storeb')},
    set(cost.keys()),
    cat=pl.constants.LpInteger
)

# Binary variables for shipping cost conditions
shipping_indicator = pl.LpVariable.dicts(
    "shipping_indicator",
    # {('land','storea'),('land', 'storeb')},
    set(cost.keys()),
    cat=pl.LpBinary
)

problem += pl.lpSum([selected_indicator[(c, s)] for c, s in selected_indicator.keys() if c == 'land']) >= 3
problem += selected_indicator[('land', 'storea')] <= 2  # qty at store a
problem += selected_indicator[('land', 'storeb')] <= 3  # qty at store b

# Add Big M constraints for shipping costs
for store in ['storea', 'storeb']:
    problem += selected_indicator[('land', store)] <= M * shipping_indicator[('land', store)]
    problem += selected_indicator[('land', store)] >= shipping_indicator[('land', store)]
    problem += shipping_indicator[('land', store)] <= 1

objective_terms = []
# item costs
objective_terms.append(selected_indicator[('land', 'storea')] * cost[('land', 'storea')])
objective_terms.append(selected_indicator[('land', 'storeb')] * cost[('land', 'storeb')])

# Add shipping costs
objective_terms.append(shipping_indicator[('land', 'storea')] * shipping_cost['storea'])
objective_terms.append(shipping_indicator[('land', 'storeb')] * shipping_cost['storeb'])


problem += pl.lpSum(objective_terms)

status = problem.solve(pl.PULP_CBC_CMD())

In [ ]:
import pulp as pl

problem = pl.LpProblem("Magic_Magic", pl.LpMinimize)

M = 1000

selected_indicator = pl.LpVariable.dicts(
    "card_store_qty",
    set(cost.keys()),
    cat=pl.constants.LpInteger
)

# Binary variables for shipping cost conditions
shipping_indicator = pl.LpVariable.dicts(
    "shipping_indicator",
    set(cost.keys()),
    cat=pl.LpBinary
)

for card in cards:
    problem += pl.lpSum([selected_indicator[(c, s)] for c, s in selected_indicator.keys() if c.rsplit('-',1)[0] == card['name']]) >= card['qty']
    
for key in stock.keys():
    problem += selected_indicator[key] <= stock[key]  

    # Add Big M constraints for shipping costs
    problem += selected_indicator[key] <= M * shipping_indicator[key]
    problem += selected_indicator[key] >= shipping_indicator[key]
    problem += shipping_indicator[key] <= 1

objective_terms = []


for key in cost.keys():
    # item costs
    objective_terms.append(selected_indicator[key] * cost[key])
    # Add shipping costs
    objective_terms.append(shipping_indicator[key] * shipping_cost[key[1]])


problem += pl.lpSum(objective_terms)

status = problem.solve(pl.PULP_CBC_CMD())

In [ ]:
print(f"Status: {pl.LpStatus[status]}")
total_cost = 0

print('\n# --------------------------------- purchases -------------------------------- #\n')
for var in selected_indicator:
    if selected_indicator[var].varValue > 0:
        print(f"{var}: {selected_indicator[var].varValue} at R{cost[var]} each --> R{selected_indicator[var].varValue * cost[var]}")
        total_cost += selected_indicator[var].varValue * cost[var]


print('\n# --------------------------------- shipping --------------------------------- #\n')
for var in shipping_indicator:
    if shipping_indicator[var].varValue > 0:
        print(f"{var}: R{shipping_indicator[var].varValue * shipping_cost[var[1]]}")
        total_cost += shipping_indicator[var].varValue * shipping_cost[var[1]]
        
print('\n# -------------------------------- breackdown -------------------------------- #\n')
for var in shipping_indicator:
    if shipping_indicator[var].varValue > 0:
        print(f"{var}: R{(selected_indicator[var].varValue * cost[var]) + shipping_indicator[var].varValue * shipping_cost[var[1]]}")

print('\n# ----------------------------------- total ---------------------------------- #\n')
print(f"Total Cost: {total_cost}")

In [3]:
from time import sleep
from MoxScraper.MoxScraper import MoxScraper
from OrderOptimizer.OrderOptimizer import OrderOptimizer
import pandas as pd

wants = [
    {
        'name': 'Generous Ent',
        'qty': 4
    },
    {
        'name': 'Oliphant',
        'qty': 4
    },
    {
        'name': 'Sneaky Snacker',
        'qty': 4
    },
    {
        'name': 'Refurbished Familiar',
        'qty': 4
    },
    {
        'name': "Trespasser's Cures",
        'qty': 4
    },
    {
        'name': 'Sneaky Snacker',
        'qty': 4
    },
    {
        'name': 'Gorilla Shaman',
        'qty': 4
    },
]

shipping_cost = {
        'D20Battleground': 90,
        'Greedy Gold': 100,
        'Sword & Board': 80,
        'Luckshack': 120,
        'Dracoti': 90,
        'The Warren': 0,
        'The Stone Dragon': 100,
        'Mirage Gaming': 100,
}

return_df = pd.DataFrame()
scraper = MoxScraper()
optimizer = OrderOptimizer()

for item in wants:
    df = scraper.format_for_retailer(scraper.scrape_mox_df(item['name']))
    if not df.empty:
        return_df = pd.concat([return_df, df], ignore_index=True)

    sleep(5)
    
report = optimizer.optimize(return_df, wants)

2024-06-14 23:58:38,794 [CUSTOM] [CustomLogger.py:26] - Querying for Generous Ent
2024-06-14 23:58:39,405 [CUSTOM] [CustomLogger.py:26] - API Response: [{'id': 946196, 'name': 'Generous Ent [The Lord of the Rings: Tales of Middle-Earth] - Near Mint', 'price': 800, 'priceRead': 'R8.00', 'link': 'https://d20battleground.co.za/products/generous-ent-the-lord-of-the-rings-tales-of-middle-earth?variant=43045349228705', 'stock': 1, 'is_foil': False, 'last_scraped': '2024-06-14T07:02:38.000000Z', 'image': 'https://cdn.shopify.com/s/files/1/0440/8944/2465/products/d59bc1a8-0d6d-5776-ada8-ba388f929a3a_868143be-e7d1-4cff-8dda-12998436da51.png?v=1705455035', 'retailer_id': 13, 'retailer_name': 'D20Battleground'}]
2024-06-14 23:58:39,423 [CUSTOM] [CustomLogger.py:26] -        id          name  price priceRead  \
0  946196  Generous Ent    800     R8.00   

                                                link  stock  is_foil  \
0  https://d20battleground.co.za/products/generou...      1    False   


KeyError: 'TopDeck'

In [ ]:
print(report)